In [ ]:
'''
====================================================
=   LIBRERÍAS NECESARIAS Y CONFIGURACIÓN INICIAL   =
====================================================
'''
import pandas as pd #Datos estructurados (Tablas, Excel, CSV, etc.)
import numpy as np #Operaciones numéricas avanzadas.
import seaborn as sns #Gráficos y visualizaciones de datos simples (líneas, barras, histogramas, etc.)
import matplotlib.pyplot as plt #Visualización de relaciones entre variables, distribuciones, mapas de calor, etc.

from sklearn.cluster import KMeans #Clase KMeans desde el módulo cluster de la librería scikit-learn
from sklearn.preprocessing import StandardScaler #Escalar o normalizar los datos numéricos antes de aplicar algoritmos de machine learning
#from mpl_toolkits.mplot3d import Axes3D #Habilitación de gráficos 3D en Matplotlib.

#Para que los gráficos se muestren directamente en el notebook
%matplotlib inline 
#Añade líneas de cuadrícula blancas al fondo del gráfico
sns.set(style="whitegrid") 

print("Importación y configuración de librerías completa!")


In [ ]:
'''
=========================================
=  VISTA INICIAL Y TIPOS DE VARIABLES   =
=========================================
'''

print("\nTipos de variables:")
print("------------------------------------")
print(df.dtypes)



In [ ]:
'''
=====================================================
=   CARGA DE DATOS Y CREACIÓN DE MODELO Y CLÚSTERS  =
=====================================================
'''
#Con la función de la librería pandas(pd) read_csv, extraemos la base de datos CSV, limpiamos y la cargamos a un DataFrame o "tabla" en Python.
df = pd.read_csv("Mall_Customers1.csv")
df_numerico = df.drop(columns=["IDCliente", "Genero"]) 

#Sirve para escalar variables numéricas y aplica el escalado a los datos del DataFrame
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numerico)

#Creción de modelo K-Means con 3 clústers o conjuntos aleatorios
algoritmo = KMeans(n_clusters=3, random_state=42)

#Entrenamiento del modelo ya con datos escalados y se asigna un clúster a cada registro
grupos = algoritmo.fit_predict(df_scaled)

#Agregar una nueva columna al DataFrame df_numerico o filtrado llamada "Num_Cluster"
df_numerico["Num_Cluster"] = grupos

# Obtener los centroides del modelo
centroides_escalados = algoritmo.cluster_centers_

# Revertir el escalado para interpretar los valores reales
centroides = scaler.inverse_transform(centroides_escalados)

# Convertir a DataFrame para mayor legibilidad e imprimirlo en pantalla
df_centroides = pd.DataFrame(centroides, columns=df_numerico.columns[:-1])  
df_centroides["Num_Cluster"] = df_centroides.index
print(df_centroides)

# Método del codo para hallar el número óptimo de clústers
inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k,random_state=42)
    kmeans.fit(df_scaled) 
    inertia.append(kmeans.inertia_)

    
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker='o', markerfacecolor='black', markeredgecolor='green')
plt.title('Método del Codo para encontrar el K óptimo')
plt.xlabel('Número de Clústeres (K)')
plt.ylabel('Inercia')
plt.grid(True)
plt.show()

In [ ]:
'''
================================
=  ESTADÍSTICAS DESCRIPTIVAS   =
================================
'''
#Mostrar un resumen estadístico de 5 ámbitos de todas las columnas del DataFrame y la cantidad de datos nulos por columnas
print("\nEstadísticas descriptivas:")
display(df.describe(include='all').sample(5)) 

print("\nValores nulos por columna:")
print(df.isnull().sum())



In [ ]:
'''
================================
=  DISTRIBUCIÓN DE VARIABLES   =
================================
'''
print("\nDistribución de variables:")

#Usando la librería Matplotlib, se crea una figura con 4 subgráficos (2 filas × 2 columnas) 
fig, axes = plt.subplots(1, 3, figsize=(18, 5))


medias_cluster = df_numerico.groupby("Num_Cluster")[["Edad", "Ingresos Anueales (k$)", "Puntuacion Gastos (1-100)"]].mean().reset_index()

# Gráfico de edad
sns.barplot(data=medias_cluster, x="Num_Cluster", y="Edad", ax=axes[0], palette="tab10")
axes[0].set_title("Edad Promedio por Clúster")
axes[0].set_ylabel("Edad")

# Gráfico de Ingresos
sns.barplot(data=medias_cluster, x="Num_Cluster", y="Ingresos Anueales (k$)", ax=axes[1], palette="tab10")
axes[1].set_title("Ingresos Promedios por Clúster")
axes[1].set_ylabel("Ingresos Anuales (k$)")

# Gráfico de Puntuación de Gastos
sns.barplot(data=medias_cluster, x="Num_Cluster", y="Puntuacion Gastos (1-100)", ax=axes[2], palette="tab10")
axes[2].set_title("Puntuación Promedio de Gastos por Clúster")
axes[2].set_ylabel("Puntuación de Gastos")

#Mostramos las gráficas
plt.tight_layout()
plt.show()

In [ ]:
'''
=================================
=  RELACIONES ENTRE VARIABLES   =
=================================
'''

#Se crea un gráfico de pares (pairplot) para ver las relaciones entre las variables numéricas, coloreando los puntos según el clúster
sns.pairplot(df_numerico, hue="Num_Cluster", palette="tab10")
plt.suptitle("Relaciones entre Variables Numéricas por Clúster", y=1.02)
plt.show()


# MATRIZ DE CORRELACION

#Agrupamos por clúster y mostramos la correlación para cada uno
#Este for recorre todos los valores únicos en la columna "Cluster" del DataFrame df_numerico, ordenados de menor a mayor.

for i in sorted(df_numerico["Num_Cluster"].unique()):
    #Crea una nueva figura de matplotlib con un tamaño de 8x6 pulgadas.
    plt.figure(figsize=(8, 6))
    #Filtra los datos del DataFrame para incluir solo las filas del clúster i y calcula la matriz de correlación con .corr()
    cluster_corr = df_numerico[df_numerico["Num_Cluster"] == i].drop(columns=["Num_Cluster"]).corr()
    #Genera un heatmap (mapa de calor) de la matriz de correlación
    sns.heatmap(cluster_corr, annot=True, cmap="coolwarm", fmt=".2f", square=True)
    #Asigna un título al gráfico indicando a qué clúster pertenece.
    plt.title(f"Matriz de Correlación - Clúster {i}")
    plt.show()